[tfrecord tutorial from TensorFlow](https://www.tensorflow.org/tutorials/load_data/tfrecord)


In [1]:
#import matplotlib.pyplot as plt
import numpy as np
#import tensorflow_datasets as tfds
import os
#from tensorflow_datasets.audio import nsynth
import tensorflow as tf
import json
from google.protobuf.json_format import MessageToJson

In [2]:
PATH_TRAIN = r"E:\nsynth233\2.3.3\train"
PATH_VALID = r"E:\nsynth233\2.3.3\valid"
TFRECORD_FILES_TRAIN = os.listdir(PATH_TRAIN )
#TFRECORD_FILES_VALIDATION = os.listdir(PATH_VALID)


In [ ]:
nsynth._AUDIO_RATE
nsynth._DESCRIPTION

In [ ]:
nsynth._SPLITS

# Inspect single TFR example

## gansynth subset

In [ ]:


dataset = tf.data.TFRecordDataset(r"E:\nsynth233\2.3.3\train\nsynth-train.tfrecord-00000-of-00128")
for d in dataset.take(1):
    ex = tf.train.SequenceExample()
    ex2 = ex.ParseFromString(d.numpy())
    gan_ex = json.loads(MessageToJson(ex))
    #print(m['features']['feature'].keys())

In [ ]:
ex3 = ex.context.feature
ex3["audio"]

In [ ]:
print(gan_ex['context']['feature'].keys())

In [ ]:
gan_ex['context']['feature']['f0/hz']

In [ ]:
gan_ex['context']['feature']['f0/confidence']

In [ ]:
gan_ex['context']['feature']['loudness/db']

In [ ]:
gan_ex['context']['feature']['f0/midi']

In [ ]:
gan_ex['context']["feature"]["audio"]

## 2.3.3 full

In [ ]:
import tensorflow as tf
import json
from google.protobuf.json_format import MessageToJson

dataset = tf.data.TFRecordDataset(r"datasets_nsynth_full_2.3.3_nsynth-test (1).tfrecord-00000-of-00008")
for d in dataset.take(1):
    ex = tf.train.SequenceExample()
    ex2 = ex.ParseFromString(d.numpy())
    m = json.loads(MessageToJson(ex))
    #print(m['features']['feature'].keys())

In [ ]:
ex3 = ex.context.feature
ex3["audio"]

In [ ]:
print(m['context']['feature'].keys())

In [ ]:
m['context']['feature']['loudness/db']

In [ ]:
m['context']["feature"]["audio"]

# Read and Write TFR Files

take some random example to write

In [ ]:
files_to_write = []

In [ ]:
for raw_record in dataset.take(2):
    files_to_write.append(raw_record)
    example_write = tf.train.SequenceExample()
    write_to_disk = raw_record

#print(write_to_disk)

(other way to represent it)

1) add the tensor file to tf.data.Dataset (more could be added)
2) write the file to some filename (it should now be visible in your folder structure)

In [ ]:


filename_example = "nsynth-valid-nico.tfrecord-111-of-32"
for idx,file in  enumerate(files_to_write):
    dataset_write = tf.data.Dataset.from_tensors(file)
    writer = tf.data.experimental.TFRecordWriter(filename_example)
    writer.write(dataset_write)




In [ ]:
dataset_write = tf.data.Dataset.from_tensor_slices(files_to_write)
writer = tf.data.experimental.TFRecordWriter(filename_example )
writer.write(dataset_write)

In [ ]:

raw_dataset_read = tf.data.TFRecordDataset(filename_example)
print(raw_dataset_read)
for raw_record_nico in raw_dataset_read.take(2):
    read_from_disk = raw_record_nico
    #print(read_from_disk)
    example_read = tf.train.SequenceExample()
    print(example_read.ParseFromString(read_from_disk.numpy()))

In [ ]:
for raw_record in raw_dataset_read.take(2):

    # extract instrument family
    example = tf.train.SequenceExample()
    example.ParseFromString(raw_record.numpy())
    example_JSON = json.loads(MessageToJson(example))
    instrument_family = example_JSON["context"]["feature"]['instrument/family']['int64List']['value']

    #['feature']['instrument/family']['int64List']['value']
    print(instrument_family)


In [ ]:

filename = f"nsynth-valid-nico.tfrecord-00-of-32"
dataset_write = tf.data.Dataset.from_tensor_slices(files_to_write)
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(dataset_write)

read the same file back in and check if it is really the same

In [ ]:
example_read.ParseFromString(read_from_disk.numpy()) == example_write.ParseFromString(write_to_disk.numpy())

so the two files represent exactly the same object !

# Sorting by instrument family

## Validation Files

In [ ]:
processed_samples_info = []
original_order_instr_fam = []
for group_index,data_group in enumerate(TFRECORD_FILES_VALIDATION):
  data_group = tf.data.TFRecordDataset(PATH_VALID + "/" + data_group)
  print(group_index)
  for raw_record in data_group:
    if group_index<=20:
      parsed_record = tf.train.SequenceExample()
      parsed_record.ParseFromString(raw_record.numpy())
      # convert raw uninformative string to JSON
      for_inst_fam = json.loads(MessageToJson(ex))
      instrument_family = for_inst_fam['context']['feature']['instrument/family']['int64List']['value']
      #instrument_type = parsed_record.features.feature['instrument_family_str'].bytes_list.value[0]
      original_order_instr_fam.append(instrument_family)
      processed_samples_info.append([group_index,raw_record,instrument_family,parsed_record])
      #print(instrument_type)
    else:
        print("finished")

In [ ]:
ex = parsed_record
m = json.loads(MessageToJson(ex))

print(m['context']['feature'].keys())

In [ ]:
print(m['context']['feature']['instrument/family']['int64List']['value'])

In [ ]:

unique_fam_values = set([fam[0] for fam in original_order_instr_fam])
print(unique_fam_values)
# looks like all validation samples belong to instr family 5, what about instr label ?

In [ ]:
instrument_families = ["bass", "brass", "flute", "guitar", "keyboard","mallet","organ","reed","string","synth_lead","vocal"]
sorted_by_instr_val = {
"bass": [],
"brass": [],
"flute" : [],
"guitar" : [],
"keyboard" : [],
"mallet" : [],
"organ" : [],
"reed" : [],
"string" : [],
"synth_lead" : [],
"vocal" : []
}



def sort_by_instrument(data, instr_dict):
    for sample in data:
      instr_fam = sample[2]
      print(instr_fam)
      instr_dict[instrument_families[int(instr_fam[0])]].append(sample)
    return instr_dict


sorted_by_instr_val  = sort_by_instrument(processed_samples_info, sorted_by_instr_val)

In [ ]:
sorted_by_instr_val["guitar"][3]

In [ ]:
guitar = sorted_by_instr_val["guitar"][3]
#guitar


In [ ]:
test_dict = {
    "bass": [],
    "brass": [],
    "flute" : [],
    "guitar" : [[0,"tensor","type","parsed"]],
    "keyboard" : [],
    "mallet" : [],
    "organ" : [],
    "reed" : [],
    "string" : [],
    "synth_lead" : [],
    "vocal" : []
}

test_guitar = test_dict["guitar"]

In [ ]:
test_guitar

## Train Files

In [3]:
# same as last one (todo ?)
def sort_raw_records_by_instrument(data):
    instrument_families = ["bass", "brass", "flute", "guitar", "keyboard","mallet","organ","reed","string","synth_lead","vocal"]
    instr_dict_train = {
        "bass": [],
        "brass": [],
        "flute" : [],
        "guitar" : [],
        "keyboard" : [],
        "mallet" : [],
        "organ" : [],
        "reed" : [],
        "string" : [],
        "synth_lead" : [],
        "vocal" : []
    }
    for sample in data:
      instr_fam = sample[1]
      #print(instr_fam)
      instr_dict_train[instrument_families[int(instr_fam[0])]].append(sample[0])
    return instr_dict_train

def writeTFRTrainFiles(instr_dict_train,sort_round):
    for key,instr_values in instr_dict_train.items():
        if instr_values:
            filename_train = f"{str(sort_round)}-train-{str(key)}-nsynth-subset-train.tfrecord-{str(sort_round)}-label-{str(key)}"
            dataset_write = tf.data.Dataset.from_tensor_slices(instr_values)
            writer = tf.data.experimental.TFRecordWriter(filename_train)
            writer.write(dataset_write)
        else:
            print(key," has no instrument values for sort round ",sort_round)
    return None

#sorted_by_instr_train  = sort_raw_records_by_instrument(processed_samples_info_train, sorted_by_instr_train)

In [7]:

upper = 20
lower = -1

#sort_round = 0,1
sort_round = 2
processed_samples_info_train = []

STARTPOSITION = 38

for source_group_index,file_in_name in enumerate(TFRECORD_FILES_TRAIN[STARTPOSITION:]):
  data_group_train = tf.data.TFRecordDataset(PATH_TRAIN + "/" + file_in_name)
  #original_order_instr_fam_train = []
  if lower < source_group_index < upper: # adjusts per round (>= act / <= act+20)
      print(source_group_index)
      for raw_record in data_group_train:
          parsed_record = tf.train.SequenceExample()
          parsed_record.ParseFromString(raw_record.numpy())
          # convert raw uninformative string to JSON and extract relevant features
          processed = json.loads(MessageToJson(parsed_record))
          instrument_family = processed['context']['feature']['instrument/family']['int64List']['value']
          #instrument_label = processed['context']['feature']['instrument/label']
          #f0_hz = processed['context']['feature']['f0/hz']
          #f0_confidence = processed['context']['feature']['f0/confidence']
          #loudness_decibel = processed['context']['feature']['loudness/db']
          #audio = processed['context']["feature"]["audio"]
          #instrument_type = parsed_record.features.feature['instrument_family_str'].bytes_list.value[0]
          #original_order_instr_fam_train.append((instrument_family,instrument_label))
          processed_samples_info_train.append([raw_record,instrument_family])
  else:
      # write files to disk and clean memory
      upper +=20
      lower +=20
      instr_dict_train = sort_raw_records_by_instrument(processed_samples_info_train)
      writeTFRTrainFiles(instr_dict_train, sort_round)
      del instr_dict_train
      del processed_samples_info_train
      print(f"round {sort_round} finished")
      sort_round += 1
      
      # continue
      for raw_record in data_group_train:
          parsed_record = tf.train.SequenceExample()
          parsed_record.ParseFromString(raw_record.numpy())
          processed = json.loads(MessageToJson(parsed_record))
          instrument_family = processed['context']['feature']['instrument/family']['int64List']['value']
          processed_samples_info_train = []
          processed_samples_info_train.append((raw_record,instrument_family))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
synth_lead  has no instrument values for sort round  2
round 2 finished
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
synth_lead  has no instrument values for sort round  3
round 3 finished
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
synth_lead  has no instrument values for sort round  4
round 4 finished
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
synth_lead  has no instrument values for sort round  5
round 5 finished
81
82
83
84
85
86
87
88
89


In [ ]:
ex = parsed_record
m = json.loads(MessageToJson(ex))

print(m['context']['feature'].keys())

In [ ]:
print(m['context']['feature']['instrument/family']['int64List']['value'])

In [ ]:

unique_fam_values = set([fam[0] for fam in original_order_instr_fam])
print(unique_fam_values)